In [ ]:
import pandas as pd
import numpy as np


country = "ES"

historical_metering = pd.read_csv(f"datasets2025/historical_metering_data_{country}.csv")

spv_ec_forecast = pd.read_excel(f"datasets2025\spv_ec00_forecasts_es_it.xlsx", sheet_name=country)

spv_ec_forecast = spv_ec_forecast.rename(columns={"Unnamed: 0": "DATETIME"})

holidays = pd.read_excel(f"datasets2025/holiday_{country}.xlsx", sheet_name="Sheet1")
holidays = holidays.rename(columns={"holiday_ES": "DATETIME"})
holidays["HOLIDAY"] = 1

rollout = pd.read_csv(f"datasets2025/rollout_data_{country}.csv")


historical_metering_cols = historical_metering.columns.tolist()
historical_metering_cols.remove("DATETIME")



#join holidays with historical metering data on DATETIME
historical_metering["DATETIME"] = pd.to_datetime(historical_metering["DATETIME"])
holidays["DATETIME"] = pd.to_datetime(holidays["DATETIME"])
historical_metering = pd.merge(historical_metering, holidays, on="DATETIME", how="left")

#merge spv_ec_forecast with historical metering data on DATETIME
historical_metering = pd.merge(historical_metering, spv_ec_forecast, on="DATETIME", how="right")

historical_metering = pd.merge(historical_metering, rollout, on="DATETIME", how="left")


historical_metering["HOLIDAY"] = historical_metering["HOLIDAY"].fillna(0)
historical_metering["HOLIDAY"] = historical_metering["HOLIDAY"].astype(int)

#extract year, month, day, hour from DATETIME
historical_metering["YEAR"] = historical_metering["DATETIME"].dt.year
historical_metering["MONTH"] = historical_metering["DATETIME"].dt.month
historical_metering["DAY"] = historical_metering["DATETIME"].dt.day
historical_metering["HOUR"] = historical_metering["DATETIME"].dt.hour
historical_metering["WEEKDAY"] = historical_metering["DATETIME"].dt.weekday
historical_metering["WEEKDAY"] = historical_metering["WEEKDAY"].astype(int)


#lagged features for 24, 48, 72, 96, 120, 144, 168 hours

for col in historical_metering_cols:
    for i in range(1, 169, 24):
        historical_metering[f"{col}_LAG_{i}"] = historical_metering[col].shift(i)

#make rolling mean for 24, 48, 72, 96, 120, 144, 168 hours
for col in historical_metering_cols:
    for i in range(24, 169, 24):
        historical_metering[f"{col}_ROLLING_MEAN_{i}"] = historical_metering[col].rolling(i).mean()

#make rolling std for 24, 48, 72, 96, 120, 144, 168 hours
for col in historical_metering_cols:
    for i in range(24, 169, 24):
        historical_metering[f"{col}_ROLLING_STD_{i}"] = historical_metering[col].rolling(i).std()

#make rolling min for 24, 48, 72, 96, 120, 144, 168 hours
for col in historical_metering_cols:
    for i in range(24, 169, 24):
        historical_metering[f"{col}_ROLLING_MIN_{i}"] = historical_metering[col].rolling(i).min()

#make rolling max for 24, 48, 72, 96, 120, 144, 168 hours
for col in historical_metering_cols:
    for i in range(24, 169, 24):
        historical_metering[f"{col}_ROLLING_MAX_{i}"] = historical_metering[col].rolling(i).max()




# --- Cyclical Time Features ---
# Hour
historical_metering["HOUR_SIN"] = np.sin(2 * np.pi * historical_metering["HOUR"] / 24)
historical_metering["HOUR_COS"] = np.cos(2 * np.pi * historical_metering["HOUR"] / 24)

# Weekday (0-6)
historical_metering["WEEKDAY_SIN"] = np.sin(2 * np.pi * historical_metering["WEEKDAY"] / 7)
historical_metering["WEEKDAY_COS"] = np.cos(2 * np.pi * historical_metering["WEEKDAY"] / 7)

# Month (1-12)
historical_metering["MONTH_SIN"] = np.sin(2 * np.pi * historical_metering["MONTH"] / 12)
historical_metering["MONTH_COS"] = np.cos(2 * np.pi * historical_metering["MONTH"] / 12)

# --- Difference Features ---
# For each original metering column, calculate the difference and percent change from the previous hour.
for col in historical_metering_cols:
    historical_metering[f"{col}_DIFF_1H"] = historical_metering[col] - historical_metering[col].shift(1)
    historical_metering[f"{col}_PCT_CHANGE_1H"] = historical_metering[col].pct_change(1)

# --- Additional Rolling Features ---
# Rolling median for 24, 48, 72, 96, 120, 144, 168 hours
for col in historical_metering_cols:
    for i in range(24, 169, 24):
        historical_metering[f"{col}_ROLLING_MEDIAN_{i}"] = historical_metering[col].rolling(i).median()

# Exponential Weighted Moving Average (EWMA) as an alternative smoothing feature
for col in historical_metering_cols:
    historical_metering[f"{col}_EWMA_24"] = historical_metering[col].ewm(span=24, adjust=False).mean()
    historical_metering[f"{col}_EWMA_48"] = historical_metering[col].ewm(span=48, adjust=False).mean()

# --- Cumulative Daily Sum ---
# Group by day (you can also use a date part if needed) to compute cumulative sum per day for each column
historical_metering["DATE"] = historical_metering["DATETIME"].dt.date
for col in historical_metering_cols:
    historical_metering[f"{col}_CUMSUM"] = historical_metering.groupby("DATE")[col].cumsum()

# Optional: Remove the temporary "DATE" column if no longer needed
historical_metering.drop("DATE", axis=1, inplace=True)


# --- Trend Extraction using Rolling Linear Regression ---
window_size = 24  # Using a 24-hour window for trend estimation

def trend_slope(x):
    # Ensure there are no missing values (or fill them appropriately)
    if np.any(np.isnan(x)):
        return np.nan
    # Fit a linear model: slope is the first coefficient
    slope = np.polyfit(range(len(x)), x, 1)[0]
    return slope

for col in historical_metering_cols:
    historical_metering[f"{col}_TREND_SLOPE_{window_size}H"] = historical_metering[col].rolling(window=window_size).apply(trend_slope, raw=True)

# --- FFT Feature Extraction using a Rolling Window ---
def dominant_frequency(x):
    # Ensure enough data points and handle missing values (fill with zero)
    if len(x) < window_size or np.any(np.isnan(x)):
        return np.nan
    # Compute the FFT of the windowed data
    fft_vals = np.fft.fft(x)
    # Compute corresponding frequency bins
    fft_freq = np.fft.fftfreq(len(x))
    # Remove the zero-frequency component (mean of the signal)
    fft_vals[0] = 0
    # Identify the index of the maximum magnitude in the FFT spectrum
    dominant_idx = np.argmax(np.abs(fft_vals))
    # Return the dominant frequency (absolute value if desired)
    return np.abs(fft_freq[dominant_idx])

for col in historical_metering_cols:
    # Use a 24-hour rolling window to compute the dominant frequency for each hour.
    historical_metering[f"{col}_FFT_DOM_FREQ_{window_size}H"] = historical_metering[col].rolling(window=window_size).apply(dominant_frequency, raw=True)




#save this dataframe to csv
historical_metering.to_csv(f"datasets2025/historical_metering_data_{country}_features.csv", index=False)

#take nan diff


<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
C:\Users\thoma\AppData\Local\Temp\ipykernel_7668\2005083316.py:9: SyntaxWarning: invalid escape sequence '\s'
  spv_ec_forecast = pd.read_excel(f"datasets2025\spv_ec00_forecasts_es_it.xlsx", sheet_name=country)
C:\Users\thoma\AppData\Local\Temp\ipykernel_7668\2005083316.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  historical_metering[f"{col}_LAG_{i}"] = historical_metering[col].shift(i)
C:\Users\thoma\AppData\Local\Temp\ipykernel_7668\2005083316.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(ax

KeyboardInterrupt: 

In [13]:
#save this dataframe to csv
historical_metering.to_csv(f"datasets2025/historical_metering_data_{country}_features.csv", index=False)

In [11]:
historical_metering

,DATETIME,VALUEMWHMETERINGDATA_customerES_1,VALUEMWHMETERINGDATA_customerES_2,VALUEMWHMETERINGDATA_customerES_5,VALUEMWHMETERINGDATA_customerES_11,VALUEMWHMETERINGDATA_customerES_19,VALUEMWHMETERINGDATA_customerES_30,VALUEMWHMETERINGDATA_customerES_31,VALUEMWHMETERINGDATA_customerES_39,VALUEMWHMETERINGDATA_customerES_40,...,VALUEMWHMETERINGDATA_customerES_297_FFT_DOM_FREQ_24H,VALUEMWHMETERINGDATA_customerES_298_FFT_DOM_FREQ_24H,VALUEMWHMETERINGDATA_customerES_300_FFT_DOM_FREQ_24H,VALUEMWHMETERINGDATA_customerES_307_FFT_DOM_FREQ_24H,VALUEMWHMETERINGDATA_customerES_311_FFT_DOM_FREQ_24H,VALUEMWHMETERINGDATA_customerES_312_FFT_DOM_FREQ_24H,VALUEMWHMETERINGDATA_customerES_313_FFT_DOM_FREQ_24H,VALUEMWHMETERINGDATA_customerES_334_FFT_DOM_FREQ_24H,VALUEMWHMETERINGDATA_customerES_335_FFT_DOM_FREQ_24H,VALUEMWHMETERINGDATA_customerES_336_FFT_DOM_FREQ_24H
0,2022-01-01 00:00:00,0.001476,0.000020,0.020160,0.021580,0.100,0.027460,0.103,0.064,0.018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01 01:00:00,0.001400,0.000024,0.019116,0.020464,0.102,0.026036,0.105,0.062,0.018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01 02:00:00,0.001360,0.000024,0.018544,0.019856,0.103,0.025264,0.106,0.058,0.019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01 03:00:00,0.001328,0.000024,0.018136,0.019416,0.109,0.024704,0.107,0.049,0.019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01 04:00:00,0.001312,0.000020,0.017920,0.019184,0.103,0.024408,0.105,0.058,0.018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28456,2025-03-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28457,2025-03-31 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28458,2025-03-31 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28459,2025-03-31 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
historical_metering

,DATETIME,VALUEMWHMETERINGDATA_customerES_1,VALUEMWHMETERINGDATA_customerES_2,VALUEMWHMETERINGDATA_customerES_5,VALUEMWHMETERINGDATA_customerES_11,VALUEMWHMETERINGDATA_customerES_19,VALUEMWHMETERINGDATA_customerES_30,VALUEMWHMETERINGDATA_customerES_31,VALUEMWHMETERINGDATA_customerES_39,VALUEMWHMETERINGDATA_customerES_40,...,VALUEMWHMETERINGDATA_customerES_335,VALUEMWHMETERINGDATA_customerES_336,HOLIDAY,spv,temp,YEAR,MONTH,DAY,HOUR,WEEKDAY
0,2022-01-01 00:00:00,0.001476,0.000020,0.020160,0.021580,0.100,0.027460,0.103,0.064,0.018,...,0.008,NaN,1,0.000000,9.2,2022,1,1,0,5
1,2022-01-01 01:00:00,0.001400,0.000024,0.019116,0.020464,0.102,0.026036,0.105,0.062,0.018,...,0.007,NaN,0,0.000000,8.9,2022,1,1,1,5
2,2022-01-01 02:00:00,0.001360,0.000024,0.018544,0.019856,0.103,0.025264,0.106,0.058,0.019,...,0.008,NaN,0,0.000000,8.8,2022,1,1,2,5
3,2022-01-01 03:00:00,0.001328,0.000024,0.018136,0.019416,0.109,0.024704,0.107,0.049,0.019,...,0.008,NaN,0,0.000000,8.6,2022,1,1,3,5
4,2022-01-01 04:00:00,0.001312,0.000020,0.017920,0.019184,0.103,0.024408,0.105,0.058,0.018,...,0.007,NaN,0,0.000000,8.1,2022,1,1,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28456,2025-03-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,8910.698834,17.2,2025,3,31,19,0
28457,2025-03-31 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,922.164988,15.1,2025,3,31,20,0
28458,2025-03-31 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0.000000,13.5,2025,3,31,21,0
28459,2025-03-31 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0.000000,12.6,2025,3,31,22,0


In [ ]:
import pandas as pd
import numpy as np
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import matplotlib.pyplot as plt

# Filter columns that contain the word 'customer'
customer_columns = [col for col in historical_metering.columns if 'customer' in col.lower()]
data = historical_metering[customer_columns]

# Convert DataFrame to a numpy array where each row represents a time series.
# If your data is organized by columns (each column is a series), then transpose the array.
X = data.values.T

# Reshape to (n_series, n_timestamps, 1) required by tslearn
X = X.reshape((X.shape[0], X.shape[1], 1))

# Normalize the time series (recommended for clustering)
scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)
X_scaled = scaler.fit_transform(X)

# Set the number of clusters (choose based on your data exploration or methods like silhouette score)
n_clusters = 3  # Adjust this value as needed

# Perform clustering using TimeSeriesKMeans with DTW as the metric
km = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", max_iter=10, random_state=0)
labels = km.fit_predict(X_scaled)

print("Cluster labels for each time series:")
print(labels)

# Visualize the cluster centers and a few example series from each cluster
for cluster in range(n_clusters):
    plt.figure(fcigsize=(10, 4))
    plt.title(f"Cluster {cluster}")
    
    # Plot the cluster center
    plt.plot(km.cluster_centers_[cluster].ravel(), 'r-', lw=2, label='Cluster Center')
    
    # Plot a few series belonging to the cluster
    for idx, series in enumerate(X_scaled[labels == cluster][:5]):  # Display up to 5 series from each cluster
        plt.plot(series.ravel(), '--', label=f'Series {idx+1}' if idx == 0 else "")
    
    plt.legend()
    plt.show()

#assign cluster labels to historical metering data
historical_metering["CLUSTER"] = labels[historical_metering["CUSTOMER"].astype(int).values]


c:\Users\thoma\miniforge3\envs\datathon2025\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)
